# Project 1

## TripAdvisor reccomandation system

In [1]:
import pandas as pd

reviews_df = pd.read_csv('./tripadvisor_hotel_reviews.csv')

print("Reviews DataFrame:")
print(reviews_df.head())

Reviews DataFrame:
                                              Review  Rating
0  nice hotel expensive parking got good deal sta...       4
1  ok nothing special charge diamond member hilto...       2
2  nice rooms not 4* experience hotel monaco seat...       3
3  unique, great stay, wonderful time hotel monac...       5
4  great stay great stay, went seahawk game aweso...       5
